# TF/IDF on data stored on Hive Assignment

> CA675

Author : Shubham Rai
Student Number : 21261161

## Cleaning Data

In [ ]:
import pandas as pd
df = pd.read_csv('stackexchange_export.csv', quotechar='"')
# Remove HTML Tags
df['Body'] = df['Body'].str.replace(r'<[^<>]*>', '', regex=True) 
# Keep letters
df["Body"] = df['Body'].str.replace('[^a-zA-Z]', ' ', regex=True)
# Remove new line characters
df["Body"] = df['Body'].replace(r'\n',' ', regex=True) 
# Export File to be imported in Hive
df.to_csv('/home/cypherhonk/cleaned_final.csv', sep=',', encoding='utf-8',index=None)
! chmod 755 cleaned_final.csv

## Get connection to Hive

In [81]:
from pyhive import hive
from tabulate import tabulate
import pandas as pd


host_name = "localhost"
port = 10000
user = "cypherhonk"
password = "941416156866692861"
database="default"

def hiveconnection(host_name, port, user,password, database):
    conn = hive.Connection(host=host_name, port=port, username=user, password=password,
                           database=database, auth='CUSTOM')
    return conn


conn = hiveconnection(host_name, port, user,password, database)
cur = conn.cursor()

## Usage example from https://github.com/dropbox/PyHive

# Task 2 & 3

### 2.2.1) Querying top 10 posts by score

In [13]:
cur.execute('select ID, Title, Score, DisplayName from stackexchange_view order by score desc limit 10')
result = cur.fetchall()
print(tabulate(result, tablefmt='orgtbl'))

| 11227809 | Why is processing a sorted array faster than processing an unsorted array? | 25933 | GManNickG       |
|   927358 | How do I undo the most recent local commits in Git?                        | 23348 | Hamza Yerlikaya |
|  2003505 | How do I delete a Git branch locally and remotely?                         | 18514 | Matthew Rankin  |
|   292357 | What is the difference between 'git pull' and 'git fetch'?                 | 12834 | pupeno          |
|   231767 | What does the "yield" keyword do?                                          | 11551 | Alex. S.        |
|   477816 | What is the correct JSON content type?                                     | 10921 | Oli             |
|   348170 | How do I undo 'git add' before commit?                                     | 10079 | paxos1977       |
|  5767325 | How can I remove a specific item from an array?                            |  9931 | Walker          |
|  6591213 | How do I rename a local Git branch?                        

### 2.2.2) The top 10 users by post score 

In [34]:
### Join data as usernames data was extracted and added post data pulling

cur.execute("""
    select
           OwnerUserId,
           DisplayName,
           sum(Score) as score
    from stackexchange_view
    group by OwnerUserId,DisplayName
    order by score desc
    LIMIT 10

""")
result = cur.fetchall()
print(tabulate(result, tablefmt='orgtbl'))

|  87234 | GManNickG       | 37672 |
|   4883 | readonly        | 28817 |
|   9951 | e-satis         | 26878 |
|   6068 | pupeno          | 25944 |
|  89904 | Hamza Yerlikaya | 24024 |
|  51816 | Joan Venge      | 23763 |
|  49153 | Ali             | 20203 |
| 179736 | TIMEX           | 19603 |
|  95592 | Matthew Rankin  | 19479 |
|  63051 | flybywire       | 19362 |


In [20]:
top_10_user_score = []
for each_item in result:
    top_10_user_score.append(each_item[0])
top_10_user_score

[87234, 4883, 9951, 6068, 89904, 51816, 49153, 179736, 95592, 63051]


### 2.2.3) The number of distinct users, who used the word “cloud” in one of their posts


In [36]:
cur.execute("""
SELECT
     COUNT(DISTINCT owneruserid) as user_count
FROM stackexchange_view
WHERE title LIKE '% cloud %' or Body LIKE '% cloud %'
""")
result = cur.fetchall()
print(tabulate(result, tablefmt='orgtbl'))

| 248 |


## TF/IDF

In [74]:
### Get data for top 10 users from above list and pull data

df = pd.read_sql(f"""
SELECT
       
       DISTINCT owneruserid,
       displayname,
       title,
       body
from stackexchange_view
WHERE owneruserid IN {tuple(top_10_user_score)}
order by owneruserid""", conn)



In [83]:
# Attach body and title fields to create one field to be 
df["text"] = df["title"] + df["body"]
top_10_username = list(df["displayname"].unique())
top_10_username

['readonly',
 'pupeno',
 'e-satis',
 'Ali',
 'Joan Venge',
 'flybywire',
 'GManNickG',
 'Hamza Yerlikaya',
 'Matthew Rankin',
 'TIMEX']

In [76]:
## References taken and custom function created. 
## Reference : https://medium.com/@cmukesh8688/tf-idf-vectorizer-scikit-learn-dbc0244a911a

from sklearn.feature_extraction.text import TfidfVectorizer


# Calculate sum() of TF-IDF and get top 10 words with highest TF-IDF and select only those columns
def calculate_tf_idf(df):
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True) # Remove Stop Words
    response = vectorizer.fit_transform(df["text"]) # Use title field for TF/IDF
    df_tfidf_sklearn = pd.DataFrame(response.toarray(),columns=vectorizer.get_feature_names())
    total_tf_idf = df_tfidf_sklearn.sum(axis = 0) # Remove sum of TF/IDF for getting top 10 most used words
    top_10_list = total_tf_idf.nlargest(10) # Get top 10 words per user
    top_10_words = list(top_10_list.index) # Get list of top 10 words
    df_tfidf_sklearn[top_10_words] # Select only top 10 words as column
    return df_tfidf_sklearn[top_10_words]


## For each of top 10 users and their top 10 words plot TF/IDF table for their correspnding comments

In [80]:
## For all 10 users create TF-IDF table for each user for their respective comments. 

for each_user in top_10_username: ## Iterate over all top 10 users
    filtered_data = df[(df['displayname']==each_user)] # Filter data only for selected user
    tf_idf_df = calculate_tf_idf(filtered_data) # pass on to above function
    print("For Username ID TF/IDF table : "+each_user) 
    tf_idf_df.insert(0, 'usernameid', each_user)# attach username ID field to dataframe
    display(tf_idf_df)
    print("******************************************************************************\n\n\n")
    

    


For Username ID TF/IDF table : readonly


,usernameid,python,use,list,table,branch,way,difference,git,process,rename
0,readonly,0.000000,0.000000,0.000000,0.000000,0.636117,0.000000,0.000000,0.202994,0.000000,0.000000
1,readonly,0.108905,0.239666,0.428552,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.350231,0.000000
4,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.203342,0.000000,0.000000
5,readonly,0.156136,0.000000,0.000000,0.000000,0.000000,0.000000,0.220471,0.000000,0.000000,0.000000
6,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.179925,0.000000,0.000000,0.000000,0.000000
7,readonly,0.000000,0.000000,0.000000,0.521156,0.000000,0.000000,0.000000,0.000000,0.000000,0.568902
8,readonly,0.127064,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,readonly,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


******************************************************************************



For Username ID TF/IDF table : pupeno


,usernameid,file,java,android,sql,git,way,like,application,data,dump
0,pupeno,0.000000,0.000000,0.000000,0.000000,0.000000,0.084849,0.000000,0.000000,0.000000,0.000000
1,pupeno,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021477,0.029489,0.000000,0.000000
2,pupeno,0.000000,0.000000,0.000000,0.000000,0.000000,0.085361,0.265579,0.000000,0.000000,0.000000
3,pupeno,0.108337,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.123813,0.000000
4,pupeno,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,pupeno,0.187244,0.000000,0.000000,0.277057,0.000000,0.109712,0.000000,0.000000,0.093622,0.000000
6,pupeno,0.116987,0.000000,0.000000,0.692403,0.000000,0.000000,0.094784,0.000000,0.116987,0.323682
7,pupeno,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.089201,0.000000,0.000000,0.000000
8,pupeno,0.000000,0.000000,0.130150,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,pupeno,0.000000,0.000000,0.000000,0.105631,0.000000,0.000000,0.072300,0.000000,0.267708,0.617252


******************************************************************************



For Username ID TF/IDF table : e-satis


...

******************************************************************************



For Username ID TF/IDF table : Matthew Rankin


,usernameid,install,pip,python,lt,gt,branch,flask,version,debugger,started
0,Matthew Rankin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.555725,0.555725,0.000000,0.000000
1,Matthew Rankin,0.000000,0.000000,0.358589,0.000000,0.000000,0.000000,0.000000,0.000000,0.495842,0.495842
2,Matthew Rankin,0.000000,0.000000,0.000000,0.000000,0.043561,0.571753,0.000000,0.000000,0.000000,0.000000
3,Matthew Rankin,0.427251,0.284834,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Matthew Rankin,0.000000,0.000000,0.000000,0.650278,0.544984,0.000000,0.000000,0.000000,0.000000,0.000000
5,Matthew Rankin,0.235094,0.211585,0.094038,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Matthew Rankin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Matthew Rankin,0.349045,0.319958,0.319958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


******************************************************************************



For Username ID TF/IDF table : TIMEX


,usernameid,python,want,user,string,gt,dictionary,lt,use,javascript,return
0,TIMEX,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000
1,TIMEX,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000
2,TIMEX,0.000000,0.130509,0.0,0.000000,0.179279,0.0,0.187309,0.000000,0.00000,0.000000
3,TIMEX,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000
4,TIMEX,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.17604,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
110,TIMEX,0.113904,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000
111,TIMEX,0.000000,0.031423,0.0,0.090197,0.000000,0.0,0.000000,0.000000,0.00000,0.048591
112,TIMEX,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.00000,0.000000
113,TIMEX,0.000000,0.000000,0.0,0.000000,0.423683,0.0,0.000000,0.076049,0.00000,0.000000


******************************************************************************



